# **Importing Titanic Dataset**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR']='/content/gdrive/MyDrive/kaggle'

In [ ]:
%cd /content/gdrive/MyDrive/kaggle

/content/gdrive/MyDrive/kaggle


In [ ]:
!ls

kaggle.json  titanic


In [ ]:
!kaggle competitions download -c titanic

  0% 0.00/34.1k [00:00<?, ?B/s]
100% 34.1k/34.1k [00:00<00:00, 3.51MB/s]


In [ ]:
!mkdir titanic

mkdir: cannot create directory ‘titanic’: File exists


In [ ]:
!mv titanic.zip titanic

In [ ]:
%cd titanic/

/content/gdrive/MyDrive/kaggle/titanic


In [ ]:
!unzip titanic.zip

Archive:  titanic.zip
replace gender_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: gender_submission.csv   
replace test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: test.csv                
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: train.csv               


#Reading csv file and storing it in dataframe 'df'

In [ ]:
#reading data from gdrive and storing it in dataframe 'df'
df = pd.read_csv('/content/gdrive/MyDrive/kaggle/titanic/train.csv')
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


Preprocessing Pipeline:
-seperate dep & indep variables (survived is dep)
-Drop Name(not needed), Cabin(too many null values), Ticket no.(not needed), passenger id(not needed)
-Impute Age w/ mean
-Turn sex into binary
-One hot encode embarked
-pclass, parch, sibsp ordinal data

# Dropping columns that are unnecessary or have too many NaN values

In [ ]:
#drop Name
df.drop(['Name'], axis = 1, inplace = True)

#Drop Cabin
df.drop(['Cabin'], axis = 1, inplace = True)

#drop ticket number
df.drop(['Ticket'], axis = 1, inplace = True)

# Imputing the 'Age' and 'Gender' columns

In [ ]:
from sklearn.impute import SimpleImputer

#impute age w/ mean
imputer = SimpleImputer(strategy='mean')
df['Age'] = imputer.fit_transform(df[['Age']])

#change male/female values to 0/1 respectively
df['Sex'] = [0 if g == 'male' else 1 for g in df['Sex']]

# Dummying the 'Embarked' column

In [ ]:
#Categorizing Embarked values
df = pd.get_dummies(df, columns = ['Embarked'])

df

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1,0,3,0,22.000000,1,0,7.2500,0,0,1
1,2,1,1,1,38.000000,1,0,71.2833,1,0,0
2,3,1,3,1,26.000000,0,0,7.9250,0,0,1
3,4,1,1,1,35.000000,1,0,53.1000,0,0,1
4,5,0,3,0,35.000000,0,0,8.0500,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,0,27.000000,0,0,13.0000,0,0,1
887,888,1,1,1,19.000000,0,0,30.0000,0,0,1
888,889,0,3,1,29.699118,1,2,23.4500,0,0,1
889,890,1,1,0,26.000000,0,0,30.0000,1,0,0


# Splitting the data into independent variables and dependent variable

In [ ]:
#independent variable
x = df.drop(['Survived'], axis = 1)

#dependent variable
y = df.Survived


# Splitting dataset into test and training data

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)

# Training the model

In [ ]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

# Using the model to make predictions

In [ ]:
predict = lr.predict(x_test)

# Making the result more readable using confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, predict), columns=['Predicted No', 'Predicted Yes'], index=['Actual No', 'Actual Yes'])

,Predicted No,Predicted Yes
Actual No,153,22
Actual Yes,37,83


In [ ]:
#Normalizing age, fare and FamilySize

from sklearn.preprocessing import Normalizer

columns_to_normalize = ['Age', 'Fare']
norm = Normalizer()
x[columns_to_normalize] = norm.fit_transform(x[columns_to_normalize])
x

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,1,3,0,0.949757,1,0,0.312988,0,0,1
1,2,1,1,0.470417,1,0,0.882444,1,0,0
2,3,3,1,0.956551,0,0,0.291564,0,0,1
3,4,1,1,0.550338,1,0,0.834942,0,0,1
4,5,3,0,0.974555,0,0,0.224148,0,0,1
...,...,...,...,...,...,...,...,...,...,...
886,887,2,0,0.901002,0,0,0.433816,0,0,1
887,888,1,1,0.535052,0,0,0.844819,0,0,1
888,889,3,1,0.784840,1,2,0.619698,0,0,1
889,890,1,0,0.654931,0,0,0.755689,1,0,0
